In [4]:
from selenium import webdriver
from selenium.webdriver.chrome import options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import os

def chromeDriver():
    if os.path.exists('.env'):
        for line in open('.env'):
            var = line.strip().split('=')
            if len(var) == 2:
                os.environ[var[0]] = var[1]
            
    # Enable JS console Output in terminal
    settingsDict = DesiredCapabilities.CHROME
    settingsDict['loggingPrefs'] = { 'browser':'ALL' }

    # Add dtm debug and omnibug
    chrome_options = Options()
    chrome_options.add_extension(os.environ['CHROME_DTM_PATH'])
    chrome_options.add_extension(os.environ['CHROME_OMNIBUG_PATH'])

    # Build the driver object
    driver = webdriver.Chrome(os.environ['CHROME_DRIVER_PATH'], chrome_options=chrome_options, desired_capabilities=settingsDict)

    return driver

In [17]:
import os
import requests
# from browser import chromeDriver
from bs4 import BeautifulSoup as bs4
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from time import sleep

In [6]:
stagingScript = "localStorage.setItem('sdsat_stagingLibrary',true);"
debugScript = "_satellite.setDebug(true);"
pageNameScript = "_satellite.getVar('Global - Pagename')"
target_url = "https://www.homepage.com"
HTML_string = "<!DOCTYPE html>..."

In [18]:
def findAllAnchors(url):

    # Use requests to get page source without using driver
    r = requests.get(url)

    # Turn the page source into soup for parsing
    soup = bs4(r.text, "html5lib")

    # Return a Set of relative anchors tags from a given
    return set([anchor for anchor in soup.find_all('a', href=True) if anchor['href'].startswith('/') and not anchor['href'].endswith(('.com', '.org', '.net'))])

In [59]:
url = "https://www.kia.com/"
r = requests.get(url)
soup = bs4(r.text, "lxml")

In [49]:
def find_all_anchors(url) : 
    result = []
    for anchor in soup.find_all('a', href = True) : 
        if anchor['href'].startswith('/') and anchor['href'].endswith('.html') : 
            result.append(anchor['href'])
            
    return result

In [50]:
all_links_main = find_all_anchors(url)

In [62]:
all_links_main

['/th/main.html',
 '/th/showroom/soul.html',
 '/th/showroom/k2500.html',
 '/th/showroom/GRANDCARNIVAL2.html',
 '/th/showroom/SOUL_EV.html',
 '/th/showroom/Stinger.html',
 '/th/showroom/soul.html',
 '/th/showroom/SOUL_EV.html',
 '/th/showroom/Stinger.html',
 '/th/showroom/k2500.html',
 '/th/showroom/suv.html',
 '/th/showroom/commercial.html',
 '/th/shopping-tools/find-a-dealer.html',
 '/th/shopping-tools/request-a-quote.html',
 '/th/shopping-tools/request-a-test-drive.html',
 '/th/shopping-tools/request-a-brochure.html',
 '/th/service/service-care.html',
 '/th/service/maintenance.html',
 '/th/service/warranty.html',
 '/th/service/roadside-assistance.html',
 '/th/service/driving-guide.html',
 '/th/service/genuine-parts.html',
 '/th/service/reference-guide.html',
 '/th/experience/about-kia.html',
 '/th/experience/brand.html',
 '/th/experience/concept-cars-techonology.html',
 '/th/experience/sponsorship.html',
 '/th/experience/innovation-story.html',
 '/th/experience/motor-show.html',
 '/t

In [61]:
for link in all_links_main : 
    test = find_all_anchors(url + link)

In [63]:
test

['/kr/main.html',
 '/kr/vehicles/all-new-morning/features.html',
 '/kr/vehicles/ray/features.html',
 '/kr/vehicles/k3/features.html',
 '/kr/vehicles/new-k5/features.html',
 '/kr/vehicles/k7/features.html',
 '/kr/vehicles/stinger/features.html',
 '/kr/vehicles/k9/microsite.html',
 '/kr/vehicles/k5phev/features.html',
 '/kr/vehicles/k5-hybrid/features.html',
 '/kr/vehicles/k7-hybrid/features.html',
 '/kr/vehicles/stonic/microsite.html',
 '/kr/vehicles/soul/features.html',
 '/kr/vehicles/soul-ev/features.html',
 '/kr/vehicles/niro/features.html',
 '/kr/vehicles/niro-phev/features.html',
 '/kr/vehicles/all-new-carens/features.html',
 '/kr/vehicles/sportage/features.html',
 '/kr/vehicles/sorento/features.html',
 '/kr/vehicles/mohave/features.html',
 '/kr/vehicles/all-new-carnival/features.html',
 '/kr/vehicles/carnival-hi-limousine/features.html',
 '/kr/vehicles/carnival-outdoor/features.html',
 '/kr/vehicles/new-k5-taxi/features.html',
 '/kr/vehicles/k7-taxi/features.html',
 '/kr/vehicles/

In [20]:
str(findAllAnchors("https://www.kia.com/th/main.html"))['anchor']

TypeError: string indices must be integers

In [10]:
def visitPage(url, relativeUrl):
    global driver
    global HTML_string

    # Visit the page
    driver.get(url+relativeUrl)

    # Turn on staging and debug (Optional)
    # driver.execute_script(stagingScript)
    # driver.execute_script(debugScript)

    # Give the page time to load
    sleep(1)

    # log events and pagename
    # format with pipe for easy parsing
    # Without try/except this will fail if _satellite is undefined
    try:
        driver.execute_script('''console.log('|' + _satellite.getVar('Global - Pagename') + '|' + s.events + '|');''')
        # parse the entirety of the JS log to get my console.logs
        # Turn it into a list for easier var extraction
        log = str([entry[u'message'] for entry in driver.get_log('browser') if entry[u'level'] == 'INFO']).split('|')

        pageName, events = str(log[1]), str(log[2])
        driver.save_screenshot(os.getcwd()+'/images/{}.png'.format(pageName))

        HTML_string += '''<tr>
                <td><a href="{}.png" target="_blank">{}</a></td>
                <td>{}</td>'''.format(os.getcwd()+ '/images/'+pageName, pageName, events)

        if "event10" in events:
            HTML_string += "<td class='green'>Good</td>"
        else:
            HTML_string +=	"<td class='red'>Bad</td>"	
        HTML_string +=	"</tr>"

    except WebDriverException:
        print("Exception on {}".format(relativeUrl))

In [9]:
def main():
    global target_url
    global HTML_string
    global driver

    relative_anchors = findAllAnchors(target_url)
    
    # Iterate the relative anchors and collect data
    for anchor in relative_anchors:
        anchorHref = str(anchor['href'])
        visitPage(target_url, anchorHref)

    # Append the closing html 
    HTML_string += '''</table></body></html>'''

    # Write the HTML to a file
    HTML_file= open("ex.html","w")
    HTML_file.write(HTML_string)
    HTML_file.close()

    driver.close()